In [1]:
using LinearAlgebra
using Dates

using Plots
using BenchmarkTools

BLAS.set_num_threads(1)

include("src/io.jl")
include("src/distances.jl")
include("src/optimizer.jl")
include("src/network.jl")
include("src/base.jl")
include("src/pretraining.jl")

# Initialize the parameters
globalParms, MCParms, NNParms, preTrainParms, systemParmsList = parametersInit()

# Initialize the input data
inputs = inputInit(globalParms, NNParms, preTrainParms, systemParmsList)
if globalParms.mode == "training"
    model, opt, refRDFs = inputs
else
    model = inputs
end;

Running ML-IMC in the training mode.


In [3]:
systemParms = systemParmsList[1];

In [3]:
traj = readXTC(systemParms)
frame = read_step(traj, 1)
coordinates = positions(frame)
box = lengths(UnitCell(frame));

In [4]:
distanceMatrix = buildDistanceMatrix(frame);

In [5]:
cutoff = 6.0 # Å
rs = 0.0 # Å 
eta = 0.1 # Å^-2
lambda = 1.0
zeta = 1.0;

In [6]:
distanceVector = distanceMatrix[:, 1];

In [7]:
function computeCosAngle(coordinates, i, j, k, distance_ij, distance_ik)::Float64
    @assert i != j && i != k && k != j 
    vector_0i = coordinates[:, i]
    vector_ij = coordinates[:, j] .- vector_0i
    vector_ik = coordinates[:, k] .- vector_0i
    cosAngle = dot(vector_ij, vector_ik) / (distance_ij * distance_ik)
    return (cosAngle)
end

computeCosAngle (generic function with 1 method)

In [8]:
@btime computeCosAngle($coordinates, 1, 461, 483, distanceVector[461], distanceVector[483])

  142.165 ns (9 allocations: 480 bytes)


0.7169325422695667

In [9]:
function G9(cosAngle, distance_ij, distance_ik, cutoff, eta, zeta, lambda=1.0, rshift=0.0)::Float64
    return (
        (1.0 + lambda * cosAngle)^zeta * 
        exp(-eta * (
                (distance_ij - rshift)^2 + 
                (distance_ik - rshift)^2)
                ) * 
        distanceCutoff(distance_ij, cutoff) * 
        distanceCutoff(distance_ik, cutoff))
end        

G9 (generic function with 3 methods)

In [10]:
function G3(cosAngle, distance_ij, distance_ik, distance_kj, cutoff, eta, zeta, lambda=1.0, rshift=0.0)::Float64
    return (
        (1.0 + lambda * cosAngle)^zeta * 
        exp(-eta * (
                (distance_ij - rshift)^2 + 
                (distance_ik - rshift)^2 +
                (distance_kj - rshift)^2)) * 
        distanceCutoff(distance_ij, cutoff) * 
        distanceCutoff(distance_ik, cutoff) *
        distanceCutoff(distance_kj, cutoff))
end        

G3 (generic function with 3 methods)

In [11]:
function G9total(i, coordinates, distanceMatrix, cutoff, eta, zeta, lambda=1.0, rshift=0.0)::Float64
    sum = 0.0
    distanceVector = distanceMatrix[:, i];
    N = length(distanceVector)
    @inbounds for k in eachindex(distanceVector)
        distance_ik = distanceVector[k]
        @inbounds for j in 1:k-1
            distance_ij = distanceVector[j]
            if 0 < distance_ij < cutoff && 0 < distance_ik < cutoff
                cosAngle = computeCosAngle(coordinates, i, j, k, distance_ij, distance_ik)
                sum += G9(cosAngle, distance_ij, distance_ik, cutoff, eta, zeta, lambda, rshift)
            end
        end
    end
    return (2.0^(1.0 - zeta) * sum)
end

G9total (generic function with 3 methods)

In [12]:
@btime G9total(5, $coordinates, $distanceMatrix, cutoff, eta, 1.0, 1.0)

  11.758 μs (333 allocations: 29.95 KiB)


0.6491223630240601

In [13]:
G9total(5, coordinates, distanceMatrix, cutoff, eta, 1.0, 1.0)

0.6491223630240601

I will implement the G3total function later (because an additional distance has to be computed)

In [15]:
@btime buildDistanceMatrix_old($frame);

  6.737 ms (2 allocations: 2.00 MiB)


In [16]:
@btime buildDistanceMatrix($frame);

  5.273 ms (3080 allocations: 12.27 MiB)


In [17]:
distanceMatrix1 = buildDistanceMatrix_old(frame)
distanceMatrix2 = buildDistanceMatrix(frame)
distanceMatrix1 == distanceMatrix2

true

In [18]:
distanceVector = distanceMatrix[:, 1];

In [19]:
@btime $distanceMatrix[:, $1];

  435.050 ns (1 allocation: 4.12 KiB)


In [31]:
coordinates = positions(frame);
r1 = coordinates[:, 1]
r2 = coordinates[:, 2]

3-element Vector{Float64}:
 15.830000638961792
 32.47000217437744
 31.760001182556152

In [35]:
distanceVector1 = distanceMatrix[:, 2];

In [37]:
distanceVector2 = computeDistanceVector(r2, coordinates, box);

In [33]:
distanceVector1 == distanceVector2

true

In [40]:
@btime updateDistance!($frame, $distanceVector, $1);

  10.216 μs (0 allocations: 0 bytes)


In [41]:
@btime computeDistanceVector($r1, $coordinates, $box);

  6.400 μs (5 allocations: 20.45 KiB)
